Создание Task в ClearML для подготовки данных

In [1]:
import pandas as pd
import numpy as np
from clearml import Task, Dataset

# Инициализируем Task для подготовки данных
task = Task.init(
    project_name='CustomerPropensityTracking',
    task_name='Data_Preparation',
    task_type=Task.TaskTypes.data_processing
)

# Логируем информацию о задаче
task.set_parameter('dataset_path', '/Users/maximvishnevskiy/ml_practice_7/transactions_diy.csv')
task.set_parameter('target_column', 'target')
task.set_parameter('target_category', 'Оборудование для сада и дачи')

# Загрузка исходных данных
logger = task.get_logger()
logger.report_text("Загрузка исходного датасета...")

df = pd.read_csv('/Users/maximvishnevskiy/ml_practice_7/transactions_diy.csv')
logger.report_text(f"Исходный датасет: {df.shape[0]} строк, {df.shape[1]} столбцов")

# Преобразуем дату
df['tr_date'] = pd.to_datetime(df['tr_date'], dayfirst=True)
df['year_month'] = df['tr_date'].dt.to_period('M')

# Создаем признак целевой категории
df['is_garden'] = (df['item_group'] == 'Оборудование для сада и дачи').astype(int)
df = df.sort_values(['client', 'tr_date'])

# Группировка по клиенту и месяцу
client_monthly = df.groupby(['client', 'year_month']).agg(
    total_amount=('amount', 'sum'),
    purchase_count=('bcode', 'nunique'),
    garden_purchase=('is_garden', 'max')
).reset_index()

# Целевая переменная: покупка в следующем месяце
client_monthly['target'] = client_monthly.groupby('client')['garden_purchase'].shift(-1)
client_monthly = client_monthly.dropna(subset=['target'])
client_monthly['target'] = client_monthly['target'].astype(int)

# Логируем статистику
logger.report_text(f"После группировки: {client_monthly.shape[0]} записей")
logger.report_text(f"Распределение target: {client_monthly['target'].value_counts().to_dict()}")

# Создаем Dataset для версионирования данных
dataset = Dataset.create(
    dataset_name='CustomerTransactions',
    dataset_project='CustomerPropensityTracking'
)

# Добавляем файлы в Dataset
dataset.add_files('/Users/maximvishnevskiy/ml_practice_7/transactions_diy.csv')
dataset.upload()

# Финализируем первую версию датасета
dataset.finalize()
logger.report_text(f"Создана версия датасета: {dataset.id}")

# Сохраняем предобработанные данные
preprocessed_path = 'preprocessed_data.csv'
client_monthly.to_csv(preprocessed_path, index=False)

# Создаем вторую версию датасета с предобработанными данными
dataset_v2 = Dataset.create(
    dataset_name='CustomerTransactions',
    dataset_project='CustomerPropensityTracking',
    parent_datasets=[dataset]
)

dataset_v2.add_files(preprocessed_path)
dataset_v2.upload()
dataset_v2.finalize()
logger.report_text(f"Создана версия датасета с предобработанными данными: {dataset_v2.id}")

# Логируем артефакт
task.upload_artifact('preprocessed_data.csv', artifact_object=preprocessed_path)
logger.report_text("Предобработанные данные сохранены как артефакт")

# Завершаем задачу
task.close()
print("Task Data_Preparation завершен!")
print(f"Dataset v1 ID: {dataset.id}")
print(f"Dataset v2 ID: {dataset_v2.id}")

ClearML Task: created new task id=7588ad37e1014433b1baf96f0c6d8542
ClearML results page: https://app.clear.ml/projects/79b0194396f14ec69257352bc2de6e92/experiments/7588ad37e1014433b1baf96f0c6d8542/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
Загрузка исходного датасета...
Исходный датасет: 1008688 строк, 7 столбцов
После группировки: 111716 записей
Распределение target: {0: 78831, 1: 32885}
ClearML results page: https://app.clear.ml/projects/e3f91fd3a0b44ae78fe3cfe96960c30a/experiments/893c6dfcf1bb43b4817e3bd216fe4da8/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/e3f91fd3a0b44ae78fe3cfe96960c30a/experiments/893c6dfcf1bb43b4817e3bd216fe4da8
Uploading dataset changes (1 files compressed to 9.89 MiB) to https://files.clear.ml
File compression and upload completed: total size 9.89 MiB, 1 chunk(s) stored (average size 9.89 MiB)
Создана версия датасета: 893c6dfcf1bb43b4817e3bd216fe4da8
ClearML results page: https:

Цель Task подготовки данных: создание отдельного эксперимента в ClearML для предобработки сырых данных, версионирования датасетов и сохранения артефактов.

Выполненные действия:

Загрузка исходного датасета transactions_diy.csv (1 008 688 строк, 7 столбцов)

Преобразование даты, создание признака is_garden для целевой категории

Агрегация данных по клиентам и месяцам

Создание целевой переменной target (покупка в следующем месяце)

Версионирование:

Dataset v1 (ID: 893c6dfcf1bb43b4817e3bd216fe4da8) — исходные данные

Dataset v2 (ID: 49e01c0361e0415aacac734065f7247c) — предобработанные данные

Сохранение предобработанных данных как артефакта Task

Скриншоты для пояснительной записки (сделайте их из веб-интерфейса ClearML):

Страница Task Data_Preparation (обзор)

Вкладка Artifacts (с прикреплённым файлом preprocessed_data.csv)

Раздел Datasets с двумя версиями датасета

Создание первого ML-эксперимента (модель 1)

In [3]:
"""
МL-эксперимент 1: Бинарная классификация с помощью RandomForest
Используется вторая версия датасета (предобработанные данные)
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import joblib
from clearml import Task, Dataset

# === 1. Инициализация Task в ClearML ===
task = Task.init(
    project_name='CustomerPropensityTracking',
    task_name='ML_Experiment_1_RandomForest',
    task_type=Task.TaskTypes.training
)

# === 2. Загрузка данных через Dataset (версия 2) ===
logger = task.get_logger()
logger.report_text("Загрузка предобработанных данных из Dataset v2...")

# Получаем Dataset по ID (версия 2 с предобработанными данными)
dataset_v2 = Dataset.get(dataset_id='49e01c0361e0415aacac734065f7247c')
dataset_path = dataset_v2.get_local_copy()

# Загружаем предобработанные данные
df = pd.read_csv(f'{dataset_path}/preprocessed_data.csv')
logger.report_text(f"Загружено {df.shape[0]} записей, {df.shape[1]} признаков")

# === 3. Подготовка признаков и целевой переменной ===
# Убираем нечисловые колонки
X = df.drop(columns=['client', 'year_month', 'target', 'year_month_str'], errors='ignore')
y = df['target']
print("Столбцы в данных:", df.columns.tolist())
print("Первые 3 строки данных:")
print(df.head(3))
# Разделение на train/val/test (уже было выполнено ранее, но сделаем явно)
# Используем временное разделение по месяцам
if 'year_month_str' in df.columns:
    # Берём последний месяц для теста
    test_mask = df['year_month_str'] == df['year_month_str'].max()
    train_val_mask = ~test_mask
    
    X_train_val = X[train_val_mask]
    y_train_val = y[train_val_mask]
    X_test = X[test_mask]
    y_test = y[test_mask]
    
    # Разделяем train/val
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val
    )
else:
    # Если нет временной метки, используем обычное разделение
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

logger.report_text(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# === 4. Логирование гиперпараметров ===
hyperparams = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'random_state': 42,
    'class_weight': 'balanced'
}

# Явно логируем гиперпараметры в ClearML
task.connect(hyperparams)
logger.report_text(f"Гиперпараметры модели: RandomForestClassifier, {hyperparams}")

# === 5. Обучение модели ===
logger.report_text("Обучение RandomForest...")
model = RandomForestClassifier(**hyperparams)
model.fit(X_train, y_train)

# === 6. Логирование метрик ===
# Прогнозы и метрики на валидации
y_val_pred = model.predict(X_val)
y_val_proba = model.predict_proba(X_val)[:, 1]

val_metrics = {
    'accuracy': accuracy_score(y_val, y_val_pred),
    'f1': f1_score(y_val, y_val_pred),
    'roc_auc': roc_auc_score(y_val, y_val_proba)
}

for metric_name, metric_value in val_metrics.items():
    logger.report_scalar(title='Validation Metrics', series=metric_name, value=metric_value, iteration=0)
    task.get_logger().report_single_value(name=f'val_{metric_name}', value=metric_value)

logger.report_text(f"Метрики на валидации: {val_metrics}")

# === 7. Сохранение модели как артефакта ===
model_path = 'randomforest_model_v1.pkl'
joblib.dump(model, model_path)
task.upload_artifact('model', model_path)
logger.report_text(f"Модель сохранена как {model_path} и загружена как артефакт")

# === 8. Тестирование на test set ===
y_test_pred = model.predict(X_test)
y_test_proba = model.predict_proba(X_test)[:, 1]

test_metrics = {
    'accuracy': accuracy_score(y_test, y_test_pred),
    'f1': f1_score(y_test, y_test_pred),
    'roc_auc': roc_auc_score(y_test, y_test_proba)
}

for metric_name, metric_value in test_metrics.items():
    logger.report_scalar(title='Test Metrics', series=metric_name, value=metric_value, iteration=0)
    task.get_logger().report_single_value(name=f'test_{metric_name}', value=metric_value)

logger.report_text(f"Метрики на тесте: {test_metrics}")

# === 9. Завершение задачи ===
task.close()
print("ML Experiment 1 завершен!")
print(f"Task ID: {task.task_id}")
print(f"ROC-AUC на валидации: {val_metrics['roc_auc']:.4f}")
print(f"ROC-AUC на тесте: {test_metrics['roc_auc']:.4f}")

Загрузка предобработанных данных из Dataset v2...
Загружено 111716 записей, 6 признаков
Столбцы в данных: ['client', 'year_month', 'total_amount', 'purchase_count', 'garden_purchase', 'target']
Первые 3 строки данных:
       client year_month  total_amount  purchase_count  garden_purchase  \
0   client100    2019-05          7299               1                1   
1  client1000    2018-09          4336               2                0   
2  client1000    2018-10           454               1                0   

   target  
0       0  
1       0  
2       0  
Train: (78201, 3), Val: (16757, 3), Test: (16758, 3)
Гиперпараметры модели: RandomForestClassifier, {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'random_state': 42, 'class_weight': 'balanced'}
Обучение RandomForest...
Метрики на валидации: {'accuracy': 0.6991704959121561, 'f1': np.float64(0.5003469124789375), 'roc_auc': np.float64(0.6727749301915409)}
Модель сохранена как randomforest_mode